# Procedimiento
1. Se detectó tarea de clasificación al encontrar los archivos *_cls.csv.
2. Se cargaron Train/Validation/Test con 13 variables predictoras.
3. Se definió una malla 3×3×3 de hiperparámetros (learning_rate, max_depth, max_iter) y se entrenaron 27 modelos (tres for anidados).
4. Métrica de coste: log_loss (menor es mejor), evaluada en train y validación.
5. Se construyó una tabla comparativa con columnas: hiperparámetros | métrica | error train | error val (hgb_resultados.csv).
6. Se eligió la mejor combinación por menor error en validación, se reentrenó sobre Train+Val y se obtuvo el error de Test.
7. Se predijo un dato nuevo inventado medianas de TestX con pequeña perturbación.

In [ ]:
# ===============================
# Experimentos con Histogram Gradient Boosting (HGB)
# Requisitos cumplidos:
# a) Usa >=3 hiperparámetros
# b) Tres valores por hiperparámetro (tres for anidados)
# c) Muestra función de costo en train y val + tabla comparativa
# d) Calcula error de test del mejor modelo
# e) Predice un dato nuevo (inventado)
# f) Incluye procedimiento y conclusiones (al final en texto)
# ===============================

import os
import numpy as np
import pandas as pd


from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.metrics import log_loss, mean_squared_error

# -------------------------------
# 1) Detectar modo (clasificación o regresión) según archivos presentes
# -------------------------------
def detect_mode():
    cls_files = [
        "TrainX_eng_cls.csv","TrainY_eng_cls.csv",
        "ValidationX_eng_cls.csv","ValidationY_eng_cls.csv",
        "TestX_eng_cls.csv","TestY_eng_cls.csv"
    ]
    reg_files = [
        "TrainX_eng_reg.csv","TrainY_eng_reg.csv",
        "ValidationX_eng_reg.csv","ValidationY_eng_reg.csv",
        "TestX_eng_reg.csv","TestY_eng_reg.csv"
    ]
    if all(os.path.exists(f) for f in cls_files):
        return "cls"
    elif all(os.path.exists(f) for f in reg_files):
        return "reg"
    else:
        raise FileNotFoundError(
            "No se encontraron todos los CSV necesarios. Asegúrate de tener los de _cls o los de _reg en esta carpeta."
        )

mode = detect_mode()
print("Modo detectado:", "Clasificación" if mode=="cls" else "Regresión")

# -------------------------------
# 2) Cargar datos
# -------------------------------
if mode == "cls":
    X_tr = pd.read_csv("TrainX_eng_cls.csv")
    y_tr = pd.read_csv("TrainY_eng_cls.csv").squeeze("columns")
    X_va = pd.read_csv("ValidationX_eng_cls.csv")
    y_va = pd.read_csv("ValidationY_eng_cls.csv").squeeze("columns")
    X_te = pd.read_csv("TestX_eng_cls.csv")
    y_te = pd.read_csv("TestY_eng_cls.csv").squeeze("columns")
else:
    X_tr = pd.read_csv("TrainX_eng_reg.csv")
    y_tr = pd.read_csv("TrainY_eng_reg.csv").squeeze("columns")
    X_va = pd.read_csv("ValidationX_eng_reg.csv")
    y_va = pd.read_csv("ValidationY_eng_reg.csv").squeeze("columns")
    X_te = pd.read_csv("TestX_eng_reg.csv")
    y_te = pd.read_csv("TestY_eng_reg.csv").squeeze("columns")

print(f"Shapes -> Train: {X_tr.shape}, Val: {X_va.shape}, Test: {X_te.shape}")
display(X_tr.head(3))

# -------------------------------
# 3) Definir malla de hiperparámetros y entrenar (tres for anidados)
#    Hiperparámetros usados (3 valores cada uno):
#    - learning_rate
#    - max_depth
#    - max_iter
# -------------------------------
param_grid = {
    "learning_rate": [0.03, 0.1, 0.3],
    "max_depth": [3, 6, None],    # None => ilimitado (controlado por max_leaf_nodes)
    "max_iter": [100, 300, 600],
}

# Otros parámetros fijos (puedes moverlos al grid si quieres ampliar la búsqueda)
fixed_params = {
    "random_state": 42,
    "l2_regularization": 0.0,
    "max_leaf_nodes": 31,
}

results = []

for lr in param_grid["learning_rate"]:
    for md in param_grid["max_depth"]:
        for it in param_grid["max_iter"]:
            params = dict(learning_rate=lr, max_depth=md, max_iter=it, **fixed_params)

            if mode == "cls":
                model = HistGradientBoostingClassifier(**params)
                model.fit(X_tr, y_tr)

                # Función de costo (error): log_loss (menor es mejor)
                p_tr = model.predict_proba(X_tr)
                err_tr = log_loss(y_tr, p_tr, labels=np.unique(y_tr))

                p_va = model.predict_proba(X_va)
                err_va = log_loss(y_va, p_va, labels=np.unique(y_tr))

                metric = "log_loss"

            else:
                model = HistGradientBoostingRegressor(**params)
                model.fit(X_tr, y_tr)

                # Función de costo (error): MSE (menor es mejor)
                yhat_tr = model.predict(X_tr)
                err_tr = mean_squared_error(y_tr, yhat_tr)

                yhat_va = model.predict(X_va)
                err_va = mean_squared_error(y_va, yhat_va)

                metric = "mse"

            results.append({
                "hyperparams": params,
                "metric": metric,
                "train_error": err_tr,
                "val_error": err_va
            })

# -------------------------------
# 4) Tabla comparativa y guardado
# -------------------------------
df_results = pd.DataFrame(results).sort_values("val_error").reset_index(drop=True)
print("Top 10 combinaciones por menor error de validación:")
display(df_results.head(10))

df_results.to_csv("hgb_resultados.csv", index=False)
print("Tabla comparativa guardada en: hgb_resultados.csv")

# -------------------------------
# 5) Elegir mejor combinación, reentrenar con Train+Val y evaluar en Test
# -------------------------------
best = df_results.iloc[0]
best_params = dict(best["hyperparams"])
print("Mejores hiperparámetros:", best_params)

X_tv = pd.concat([X_tr, X_va], axis=0).reset_index(drop=True)
y_tv = pd.concat([pd.Series(y_tr), pd.Series(y_va)], axis=0).reset_index(drop=True)

if mode == "cls":
    best_model = HistGradientBoostingClassifier(**best_params)
    best_model.fit(X_tv, y_tv)

    # Probabilidades en TEST
    p_te = best_model.predict_proba(X_te)

    # ¡Aquí va y_te, no y_tv.unique()!
    test_error = log_loss(y_te, p_te, labels=np.unique(y_tv))
    print(f"Error de TEST (log_loss): {test_error:.6f}")

else:
    best_model = HistGradientBoostingRegressor(**best_params)
    best_model.fit(X_tv, y_tv)
    yhat_te = best_model.predict(X_te)
    test_error = mean_squared_error(y_te, yhat_te)
    print(f"Error de TEST (MSE): {test_error:.6f}")


# -------------------------------
# 6) Predicción de un dato nuevo (inventado)
#    Creamos un vector con medianas de TestX y una leve perturbación
# -------------------------------
new_sample = X_te.median(numeric_only=True).to_frame().T
for c in new_sample.columns[: min(5, new_sample.shape[1])]:
    new_sample[c] = new_sample[c] * (1 + 0.01)  # +1% para no ser exactamente la mediana

print("Ejemplo de dato nuevo (primeras columnas):")
display(new_sample.iloc[:, : min(8, new_sample.shape[1])])

if mode == "cls":
    proba_new = best_model.predict_proba(new_sample)[0]
    pred_new = best_model.predict(new_sample)[0]
    print("Probabilidades por clase:", proba_new)
    print("Predicción de clase para el dato nuevo:", pred_new)
else:
    pred_new = best_model.predict(new_sample)[0]
    print("Predicción (valor continuo) para el dato nuevo:", float(pred_new))


Modo detectado: Clasificación
Shapes -> Train: (25621, 13), Val: (6406, 13), Test: (8007, 13)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.735670,1.198458,-0.217228,0.255213,1.606725,-1.332239,-0.143193,0.796480,0.425918,2.264962,-1.070297,-0.093718,1.480903
1,-1.142184,1.309113,-0.306729,1.917294,-1.034587,0.294255,-0.328716,-0.844322,-0.904250,1.473451,-1.373317,0.645823,0.565404
2,-1.280064,-1.410306,-1.121035,-1.376182,-1.345795,-1.074051,-0.302140,-0.909713,-0.860857,0.360795,-1.370034,0.533145,0.134861


Top 10 combinaciones por menor error de validación:


,hyperparams,metric,train_error,val_error
0,"{'learning_rate': 0.03, 'max_depth': None, 'ma...",log_loss,0.293521,0.440010
1,"{'learning_rate': 0.03, 'max_depth': None, 'ma...",log_loss,0.307976,0.441087
2,"{'learning_rate': 0.1, 'max_depth': None, 'max...",log_loss,0.300029,0.442757
3,"{'learning_rate': 0.1, 'max_depth': None, 'max...",log_loss,0.300029,0.442757
4,"{'learning_rate': 0.1, 'max_depth': None, 'max...",log_loss,0.300029,0.442757
5,"{'learning_rate': 0.03, 'max_depth': 6, 'max_i...",log_loss,0.296296,0.443672
6,"{'learning_rate': 0.1, 'max_depth': 6, 'max_it...",log_loss,0.283311,0.445433
7,"{'learning_rate': 0.1, 'max_depth': 6, 'max_it...",log_loss,0.283311,0.445433
8,"{'learning_rate': 0.03, 'max_depth': 6, 'max_i...",log_loss,0.326888,0.445992
9,"{'learning_rate': 0.1, 'max_depth': 6, 'max_it...",log_loss,0.313091,0.447073


Tabla comparativa guardada en: hgb_resultados.csv
Mejores hiperparámetros: {'learning_rate': 0.03, 'max_depth': None, 'max_iter': 600, 'random_state': 42, 'l2_regularization': 0.0, 'max_leaf_nodes': 31}
Error de TEST (log_loss): 0.429900
Ejemplo de dato nuevo (primeras columnas):


,0,1,2,3,4,5,6,7
0,-1.111941,-0.799436,-0.297185,-0.126462,-0.173063,0.187472,-0.4214,-0.017099


Probabilidades por clase: [0.04999049 0.88431264 0.06569688]
Predicción de clase para el dato nuevo: 2


# Conclusiones
1. El mejor modelo fue learning_rate=0.03, max_iter=600, max_depth=None.
2. Generaliza bien: val_log_loss ≈ 0.440 y test_log_loss = 0.430 (muy próximos).
3. Hay un ligero sobreajuste (train más bajo que val). Se podría mitigar con mayor regularización (l2_regularization), control de hojas (max_leaf_nodes, min_samples_leaf) o learning_rate aún menor con más iteraciones.
4. La predicción del dato nuevo resultó en clase 2; las probabilidades muestran alta confianza para esa clase.
5. Posibles mejoras: ampliar la búsqueda de hiperparámetros, probar distintas semillas, y agregar métricas y análisis por clase (confusión, ROC/PR) para diagnóstico fino.